In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import os

In [2]:
import skimage
from skimage.transform import resize

In [3]:
imageSize = 224
target_dims = (imageSize, imageSize, 3)
num_classes = 29

train_len = 50
train_dir = r'C:/Users/vatsa/OneDrive/Desktop/Covid-19-X-Ray-Recognition/dataset/'

def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = np.empty((train_len, imageSize, imageSize, 3), dtype=np.float32)
    y = np.empty((train_len,), dtype=np.int)
    cnt = 0

    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['normal']:
                label = 0
            elif folderName in ['covid']:
                label = 1
                
            for image_filename in os.listdir(folder + folderName):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file).reshape((-1, imageSize, imageSize, 3))
                    
                    X[cnt] = img_arr
                    y[cnt] = label
                    cnt += 1
#                     X.append(img_arr)
#                     y.append(label)
#     X = np.asarray(X)
#     y = np.asarray(y)
    return X,y
X_data, y_data = get_data(train_dir)

In [4]:
X_data.shape

(50, 224, 224, 3)

In [5]:
y_data.shape

(50,)

In [6]:
from keras.utils import to_categorical

Using TensorFlow backend.
c:\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py

In [7]:
Y_data = to_categorical(y_data)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3,
                                                    stratify=Y_data, random_state=42)

In [10]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(35, 224, 224, 3) (15, 224, 224, 3)
(35, 2) (15, 2)


In [11]:
del X_data, y_data, Y_data

### Creating The Model

In [12]:
from keras.layers import Convolution2D, Dense, Dropout, Flatten, Activation, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
BS = 8

In [14]:
model = Sequential()

model.add(Convolution2D(32, (5, 5), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Convolution2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(2, activation='sigmoid'))

model.summary()





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 220, 220, 32)      2432      
_________________________________________________________________
activation_1 (Activation)    (None, 220, 220, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      18496     
_________________________________________________________________
activation_2 (Activation)    (None, 108, 108, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 52, 52, 64)   

In [15]:
from keras.layers import Conv2D, MaxPool2D

model = Sequential()

model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())

model.add(Dense(units=4096,activation="relu"))

model.add(Dense(units=4096,activation="relu"))

model.add(Dense(units=2, activation="softmax"))

model.summary()

In [16]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
from keras.preprocessing.image import ImageDataGenerator

In [18]:
trainAug = ImageDataGenerator(rotation_range=15,
                              fill_mode="nearest")

In [19]:
testAug = ImageDataGenerator(rotation_range=10,
                              fill_mode="nearest")

In [20]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [21]:
model.fit_generator(trainAug.flow(X_train, y_train, batch_size=BS, shuffle=True),
                    steps_per_epoch=len(X_train)//BS,
                    validation_data=testAug.flow(X_test, y_test, batch_size=BS, shuffle=False),
                    validation_steps=len(X_test)//BS,
                    epochs=25,
                    verbose=2,
                    callbacks=[early_stop])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/25
 - 3s - loss: 0.8012 - acc: 0.5370 - val_loss: 0.6987 - val_acc: 0.3750
Epoch 2/25
 - 0s - loss: 0.6903 - acc: 0.5000 - val_loss: 0.6788 - val_acc: 0.5714
Epoch 3/25
 - 0s - loss: 0.6832 - acc: 0.5787 - val_loss: 0.7065 - val_acc: 0.3750
Epoch 4/25
 - 0s - loss: 0.6867 - acc: 0.5833 - val_loss: 0.6595 - val_acc: 1.0000
Epoch 5/25
 - 0s - loss: 0.6550 - acc: 0.9028 - val_loss: 0.5820 - val_acc: 1.0000
Epoch 6/25
 - 0s - loss: 0.6304 - acc: 0.7407 - val_loss: 0.4447 - val_acc: 1.0000
Epoch 7/25
 - 0s - loss: 0.4257 - acc: 0.8704 - val_loss: 0.3426 - val_acc: 0.8750
Epoch 8/25
 - 0s - loss: 0.4238 - acc: 0.8611 - val_loss: 0.3645 - val_acc: 0.8571
Epoch 9/25
 - 0s - loss: 0.2027 - acc: 0.9062 - val_loss: 0.0160 - val_acc: 1.0000
Epoch 10/25
 - 0s - loss: 0.2064 - acc: 0.9028 - val_loss: 0.0813 - val_acc: 1.0000
Epoch 11/25
 - 0s - loss: 0.3101 - acc: 0.8519 - val_loss: 0.0419 - val_ac

In [ ]:
del model